<a href="https://colab.research.google.com/github/Naduni366/Deep-Learning-Mini-Project-03/blob/main/DL3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import tensorflow as tf
import string
import re
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import Input, Embedding, Dense, Masking
from keras.layers import Attention, LayerNormalization, Dropout
from keras.optimizers import Adam


In [ ]:
from google.colab import drive
df = drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import os

os.chdir("/content/drive/My Drive/dl3/")
print(os.getcwd())




/content/drive/My Drive/dl3


In [ ]:
file1_path = '/content/drive/My Drive/dl3/training_data_src.txt'
file2_path = '/content/drive/My Drive/dl3/training_data_tgt.txt'

# Read the contents of the text files
with open(file1_path, 'r', encoding='utf-8') as file:
    training_data_src = file.readlines()

with open(file2_path, 'r', encoding='utf-8') as file:
    training_data_tgt = file.readlines()

# Optionally, you can print the first few lines of each dataset to verify the contents
print("Dataset 1:")
print(training_data_src[:5])  # Print the first 5 lines
print("Dataset 2:")
print(training_data_tgt[:5])  # Print the first 5 lines

Dataset 1:
['Then what do I have to take out, key, bucket or well.\n', 'Keys. - Yes\n', 'In the manor. - Yes.\n', 'Right now. - Yes.\n', 'In this darkness.\n']
Dataset 2:
['එතකොට මට මොනවාද එලියට ගන්න තියෙන්නේ? යතුරද, පනිට්ටුවද, වෙලද?  \n', 'යතුරු -  ඔව්\n', 'මාලිගාවෙ? - ඔව් \n', 'මේ දැන් - ඔව් \n', 'මේ අඳුරේ?\n']


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Tokenize the source sequences
src_tokenizer = Tokenizer(filters='')
src_tokenizer.fit_on_texts(training_data_src)
src_vocab_size = len(src_tokenizer.word_index) + 1
src_max_length = max([len(sequence.split()) for sequence in training_data_src])

# Tokenize the target sequences
tgt_tokenizer = Tokenizer(filters='')
tgt_tokenizer.fit_on_texts(training_data_tgt)
tgt_vocab_size = len(tgt_tokenizer.word_index) + 1
tgt_max_length = max([len(sequence.split()) for sequence in training_data_tgt])

# Prepare training data
encoder_input_data = src_tokenizer.texts_to_sequences(training_data_src)
decoder_input_data = tgt_tokenizer.texts_to_sequences(training_data_tgt)
decoder_target_data = [[token for token in sequence[1:]] + [0] for sequence in decoder_input_data]

encoder_input_data = pad_sequences(encoder_input_data, maxlen=src_max_length, padding='post')
decoder_input_data = pad_sequences(decoder_input_data, maxlen=tgt_max_length, padding='post')
decoder_target_data = pad_sequences(decoder_target_data, maxlen=tgt_max_length, padding='post')

# Define the model architecture
latent_dim = 256

encoder_inputs = tf.keras.layers.Input(shape=(src_max_length,))
encoder_embedding = tf.keras.layers.Embedding(src_vocab_size, latent_dim, mask_zero=True)(encoder_inputs)
encoder_outputs, state_h, state_c = tf.keras.layers.LSTM(latent_dim, return_state=True)(encoder_embedding)
encoder_states = [state_h, state_c]

decoder_inputs = tf.keras.layers.Input(shape=(tgt_max_length,))
decoder_embedding = tf.keras.layers.Embedding(tgt_vocab_size, latent_dim, mask_zero=True)(decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = tf.keras.layers.Dense(tgt_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Compile the model
model = tf.keras.models.Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

# Train the model
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=64,
          epochs=20,
          validation_split=0.2)

# Save the model
model.save('/content/drive/My Drive/dl3/translator_model')


Epoch 1/20
1/1 [==============================] - 13s 13s/step - loss: 4.7539 - val_loss: 4.7533
Epoch 2/20
1/1 [==============================] - 0s 157ms/step - loss: 4.7388 - val_loss: 4.7513
Epoch 3/20
1/1 [==============================] - 0s 167ms/step - loss: 4.7232 - val_loss: 4.7491
Epoch 4/20
1/1 [==============================] - 0s 156ms/step - loss: 4.7063 - val_loss: 4.7466
Epoch 5/20
1/1 [==============================] - 0s 147ms/step - loss: 4.6871 - val_loss: 4.7436
Epoch 6/20
1/1 [==============================] - 0s 154ms/step - loss: 4.6646 - val_loss: 4.7398
Epoch 7/20
1/1 [==============================] - 0s 150ms/step - loss: 4.6368 - val_loss: 4.7349
Epoch 8/20
1/1 [==============================] - 0s 158ms/step - loss: 4.6010 - val_loss: 4.7281
Epoch 9/20
1/1 [==============================] - 0s 154ms/step - loss: 4.5517 - val_loss: 4.7176
Epoch 10/20
1/1 [==============================] - 0s 168ms/step - loss: 4.4789 - val_loss: 4.7004
Epoch 11/20
1/1 [===

In [ ]:
# Inference Models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_embedding_inf = decoder_embedding(decoder_inputs)

decoder_outputs_inf, state_h_inf, state_c_inf = decoder_lstm(
    decoder_embedding_inf, initial_state=decoder_states_inputs)
decoder_states_inf = [state_h_inf, state_c_inf]
decoder_outputs_inf = decoder_dense(decoder_outputs_inf)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs_inf] + decoder_states_inf)

# Translation Function
def translate_sentence(input_sentence):
    input_sequence = pad_sequences(src_tokenizer.texts_to_sequences([input_sentence]), maxlen=src_max_length, padding='post')
    states_value = encoder_model.predict(input_sequence)
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = tgt_tokenizer.word_index['<start>']  # Assuming '<start>' token is used

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = tgt_tokenizer.index_word[sampled_token_index]

        if sampled_word == '<end>' or len(decoded_sentence.split()) > tgt_max_length:
            stop_condition = True
        else:
            decoded_sentence += ' ' + sampled_word

        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index
        states_value = [h, c]

    return decoded_sentence.strip()

# Test the model
input_sentence = "Hello, how are you?"
translated_sentence = translate_sentence(input_sentence)
print("Translated sentence:", translated_sentence)

TypeError: 'KerasTensor' object is not callable